In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
%matplotlib inline
from scipy.stats import norm, skew
from scipy import stats
import scipy

In [ ]:
train_df = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
train_df.info()

In [ ]:
train_df.sample(10)

In [ ]:
train_df.describe()

In [ ]:
print(train_df['SalePrice'].describe())

In [ ]:
#DROP ID COLUMN
train_ID = train_df['Id']
test_ID = test_df['Id']
train_df.drop("Id", axis=1, inplace=True)
test_df.drop("Id", axis=1, inplace=True)

In [ ]:
#Histogram of sale prices
plt.hist(train_df.SalePrice,bins=50)
plt.xlabel('Price')
plt.ylabel('Number of the houses')

In [ ]:
#plot the LotFrontage
#plt.scatter(train_df.iloc[:,3],"SalePrice")

#plt.ylabel('Sale Price')
#plt.xlabel('LotFrontage')

In [ ]:
obj=train_df.isnull().sum()
for key,value in obj.iteritems():
    print(key,",",value)

In [ ]:
#Numerical feature correlation
plt.figure(figsize = (20,10))
internal_chars = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea',
                 'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF',
                 'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','SalePrice']
corrmat = train_df[internal_chars].corr()
sns.heatmap(corrmat,square=False,linewidths=.5,annot=True)

In [ ]:
print(corrmat["SalePrice"].sort_values(ascending=False))

looks like GrLivArea, GarageCars, GarageArea, TotalBsmtSF, 1stFlrSF, FullBath features have high correlation to the saleprice.

****1.Data Processing****

1.1 Outliers

In [ ]:
#Deal with outliers in GrLivArea
sns.scatterplot(x='GrLivArea',y='SalePrice',data=train_df)

In [ ]:
train_df[(train_df['GrLivArea']>4000) & (train_df['SalePrice']<400000)]

In [ ]:
train_df[(train_df['GrLivArea']>4000) & (train_df['SalePrice']<400000)].index

In [ ]:
ind_drop = train_df[(train_df['GrLivArea']>4000) & (train_df['SalePrice']<400000)].index

In [ ]:
train_df = train_df.drop(ind_drop,axis=0)

In [ ]:
sns.scatterplot(x='GrLivArea',y='SalePrice',data=train_df)

1.2 Target Variable

In [ ]:
sns.displot(train_df['SalePrice'])
# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train_df['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get the probability plot
fig = plt.figure()
res = stats.probplot(train_df['SalePrice'], plot=plt)
plt.show()

The target variable is right skewed. As (linear) models love normally distributed data , we need to transform this variable and make it more normally distributed.

1.3 Log-transformation of the target

In [ ]:
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

#check the new distribution
sns.distplot(train_df['SalePrice'])

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train_df['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the probability plot
fig = plt.figure()
res = stats.probplot(train_df['SalePrice'], plot=plt)
plt.show()

The skew seems now corrected and the data appears more normally distributed.



**2.Feature engineering**

In [ ]:
#Concatenate the train and test dataset
y_train = train_df.SalePrice.values
all_df = pd.concat((train_df,test_df)).reset_index(drop=True)
all_df.drop(["SalePrice"],axis=1,inplace=True)
#print("all_df size: {}".format(all_df.shape))
all_df.info()

2.1 Missing values

In [ ]:
#Check the missing value ratio of each feature
all_df_na = (all_df.isnull().sum() / len(all_df)) * 100
all_df_na = all_df_na.drop(all_df_na[all_df_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_df_na})
missing_data.head(20)

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=all_df_na.index, y=all_df_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

2.2 Imputing missing values

* PoolQC

In [ ]:
# NA means "No pool" , so we can fill in with none
all_df["PoolQC"].unique()
all_df["PoolQC"] = all_df["PoolQC"].fillna("None")

* MiscFeature

In [ ]:
# NA means "no misc feature"
all_df["MiscFeature"].unique()
all_df["MiscFeature"] = all_df["MiscFeature"].fillna("None")

* Alley

In [ ]:
all_df["Alley"].unique()
all_df["Alley"] = all_df["Alley"].fillna("None")

* Fence

In [ ]:
all_df["Fence"] = all_df["Fence"].fillna("None")

* FireplaceQu

In [ ]:
all_df["FireplaceQu"] = all_df["FireplaceQu"].fillna("None")

* LotFrontage

In [ ]:
#LotFrontage: Linear feet of street connected to property,Neighborhood: Physical locations within Ames city limits,fill in missing values by the median LotFrontage of the neighborhood
all_df["LotFrontage"] = all_df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

* GarageType, GarageFinish, GarageQual and GarageCond

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_df[col] = all_df[col].fillna('None')

* GarageYrBlt, GarageArea and GarageCars

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_df[col] = all_df[col].fillna(0)

* BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBath(num)

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_df[col] = all_df[col].fillna(0)

* BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 and BsmtFinType2(cat)

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_df[col] = all_df[col].fillna('None')

* MasVnrArea and MasVnrType

In [ ]:
all_df["MasVnrType"] = all_df["MasVnrType"].fillna("None")
all_df["MasVnrArea"] = all_df["MasVnrArea"].fillna(0)

* MSZoning (The general zoning classification)

In [ ]:
all_df.MSZoning.value_counts()#The majority of the data are RL, so we can fill in the missing data with mode

In [ ]:
all_df['MSZoning'] = all_df['MSZoning'].fillna(all_df['MSZoning'].mode()[0])

* Utilities


In [ ]:
all_df.Utilities.value_counts() #This feature won't help with the predicting so we can just delete it

In [ ]:
all_df = all_df.drop(['Utilities'], axis=1)

* Functional

In [ ]:
#data description says NA means typical
all_df["Functional"] = all_df["Functional"].fillna("Typ")

* Electrical

In [ ]:
all_df.Electrical.value_counts()

In [ ]:
all_df['Electrical'] = all_df['Electrical'].fillna(all_df['Electrical'].mode()[0])

* KitchenQual

In [ ]:
all_df['KitchenQual'] = all_df['KitchenQual'].fillna(all_df['KitchenQual'].mode()[0])

* Exterior1st and Exterior2nd

In [ ]:
#only has one missing value so just fill in with mode
all_df['Exterior1st'] = all_df['Exterior1st'].fillna(all_df['Exterior1st'].mode()[0])
all_df['Exterior2nd'] = all_df['Exterior2nd'].fillna(all_df['Exterior2nd'].mode()[0])

* SaleType

In [ ]:
all_df['SaleType'] = all_df['SaleType'].fillna(all_df['SaleType'].mode()[0])

* MSSubClass

In [ ]:
all_df['MSSubClass'] = all_df['MSSubClass'].fillna("None")

In [ ]:
#double check the missing values
all_df_na = (all_df.isnull().sum() / len(all_df)) * 100
all_df_na = all_df_na.drop(all_df_na[all_df_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_df_na})
missing_data.head()

2.3 Data Correlation

In [ ]:
#Correlation map to see how features are correlated with SalePrice
corrmat = train_df.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

2.4 Transform the datatype of numerical values that are really categorical

In [ ]:
all_df['MSSubClass'] = all_df['MSSubClass'].apply(str)
all_df['OverallCond'] = all_df['OverallCond'].astype(str)

all_df['YrSold'] = all_df['YrSold'].astype(str)
all_df['MoSold'] = all_df['MoSold'].astype(str)

2.5 Label encoding the categorical variables which have ordering set

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for f in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_df[f].values)) 
    all_df[f] = lbl.transform(list(all_df[f].values))
        
print('Shape all_df: {}'.format(all_df.shape))

2.6 Creat new features

In [ ]:
#Add total area
all_df['TotalSF'] = all_df['TotalBsmtSF'] + all_df['1stFlrSF'] + all_df['2ndFlrSF']

In [ ]:
#Add skewed features
numeric_feats = all_df.dtypes[all_df.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_df[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

2.7 Box Cox Transformation of (highly) skewed features

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_df[feat] = boxcox1p(all_df[feat], lam)
    

2.8 Get dummy categorical features

In [ ]:
all_df = pd.get_dummies(all_df)
all_df.info()

**3.Modelling**

In [ ]:
#Get new train and test data
ntrain = train_df.shape[0]
ntest = test_df.shape[0]
train = all_df[:ntrain]
test = all_df[ntrain:]

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

3.1 Define cross validation strategy

In [ ]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

**Base models**

* LASSO Regression :

In [ ]:
#make robust to outliers
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

* Elastic Net Regression :

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

* Kernel Ridge Regression :

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

* Gradient Boosting Regression :

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)


* XGBoost :

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

* LightGBM :

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

model_lgb2 = lgb.LGBMRegressor(objective='regression',num_leaves=15,
                              learning_rate=0.5, n_estimators=70,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

**Stacking models**

Averaging base models:

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #The predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [ ]:
class WeightedModel():
    def __init__(self, M1, M2):
        self.M1 = M1
        self.M2 = M2
        
    def fit(self,X,y):
        for model in (self.M1,self.M2):
            model.fit(X,y)
        return self
    
    def predict(self,X):
        prediction1 = self.M1.predict(X)
        prediction2 = self.M2.predict(X)
        return prediction1*0.3+prediction2*0.7

In [ ]:
#Averaged base models score

averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

**Ensembling StackedRegressor, XGBoost and LightGBM**

In [ ]:
#define a rmsle function
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

**Final training and prediction**

In [ ]:
#StackedRegressor:
stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
#XGboost:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
#LightGBM:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15


In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)